In [1]:
import sys
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python/")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/")
sys.path.append("/home/rossidaniel/Dropbox")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python")

In [2]:
import Bin_Packing_Py as BP
import BP_print
import BP_plot
import BP_utilities
import HA_routines
import BP_instances

import matplotlib
%matplotlib nbagg
matplotlib.rcParams['figure.figsize'] = [6,3.5]

from ortools.linear_solver import pywraplp
import random
import time
import csv

import pandas as pd
from mail import send_mail
from Test_BP_G import euristic
import multiprocessing

In [3]:
def tc_sfida(ist):
    item_list = []
    bin_list = []

    bin_list.append(BP.Bin(2.70, 100, 2.45))
    bin_list.append(BP.Bin(0, 0, 0))

    n_items =random.randint(3,10)

    for i in range(n_items):
        w=random.uniform(ist.width.a,ist.width.b)
        d=random.uniform(ist.depth.a,ist.depth.b)
        h=random.uniform(0.5,1.5)
        item = BP.Item(i,h,d,w)
        item.stackable = random.randint(0,0)
        item.order_id = random.randint(0,3)
        item.weight = 4
        item_list.append(item)

    return item_list, bin_list

In [4]:
class istanza:
    def __init__(self,int1,int2,ist_id):
        self.index=ist_id
        self.width=int1
        self.depth=int2
        
class intervallo:
    def __init__(self,a,b):
        self.a=a
        self.b=b

In [5]:
class model_bin_packing:
    def __init__(self,item_list,bin_list):
        self.bp = BP.BinPack()
        self.bp.items.extend(item_list)
        self.bp.bins.extend(bin_list)
        
    def buildModel(self,amount_time=300000):
        solver = pywraplp.Solver('Model2DRS', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        #importo dimensioni dei pacchi e del camion
        n=len(self.bp.items)

        w =[self.bp.items[i].width for i in range(n)]
        d =[self.bp.items[i].depth for i in range(n)]
        h =[self.bp.items[i].height for i in range(n)]
        
        W =self.bp.bins[0].width
        D =solver.NumVar(0,sum([max(w[i],d[i]) for i in range(n)]),"D")
        H =self.bp.bins[0].height

        Mw=W+max(max(w),max(d))
        Md=self.bp.bins[0].depth+max(max(w),max(d))
        Mh=self.bp.bins[0].height+max(h)
                
        # definisco le variabili
        l =[[solver.BoolVar("l%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        b =[[solver.BoolVar("b%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        t =[[solver.IntVar(0,self.bp.items[i].stackable,"t%d%d" % (i,j)) for j in range(n)] for i in range(n)]
        f =[[solver.IntVar(0,self.bp.items[i].stackable,"f%d%d" % (i,j)) for j in range(n)] for i in range(n)]
        k =[[solver.BoolVar("k%d%d" % (i,j)) for j in range(n)] for i in range(n)]
        r =[solver.BoolVar("r%d" % (i)) for i in range(n)]

        x =[solver.NumVar(0,solver.infinity(),"x%d" % i) for i in range(n)]
        y =[solver.NumVar(0,solver.infinity(),"y%d" % i) for i in range(n)]
        z =[solver.NumVar(0,solver.infinity(),"z%d" % i) for i in range(n)]
        
        #definisco i constraints
        for i in range(n):
            for j in range(n):
                if(i < j):
                    solver.Add(k[i][j] <= l[i][j] + l[j][i] + b[i][j] + b[j][i])
                    solver.Add(2 * k[i][j] >= l[i][j] + l[j][i] + b[i][j] + b[j][i])
                    solver.Add(1 - k[i][j] == t[i][j] + t[j][i])
                if(i != j):
                    solver.Add(x[i] - x[j] + Mw * l[i][j] <= Mw - w[i]*(1-r[i]) - d[i]*r[i])
                    solver.Add(y[i] - y[j] + Md * b[i][j] <= Md - d[i]*(1-r[i]) - w[i]*r[i])
                    
                    solver.Add(x[i] - x[j] <= Mw * (1 - f[i][j]))
                    solver.Add(y[i] - y[j] <= Md * (1 - f[i][j]))
                    
                    solver.Add(x[i] - x[j] + w[i]*(1-r[i]) + d[i]*r[i] - w[j]*(1-r[j]) - d[j]*r[j] >= - Mw * (1 - f[i][j]))
                    solver.Add(y[i] - y[j] + d[i]*(1-r[i]) + w[i]*r[i] - d[j]*(1-r[j]) - w[j]*r[j] >= - Md * (1 - f[i][j]))

                    solver.Add(z[i] - z[j] + Mh * t[i][j] <= Mh - h[i])
                    solver.Add(z[i] - z[j] - Mh * f[i][j] >= - Mh - h[i])
                    solver.Add(f[i][j] <= t[i][j])

            solver.Add(x[i] + w[i]*(1-r[i]) + d[i]*r[i] <= W)
            solver.Add(y[i] + d[i]*(1-r[i]) + w[i]*r[i] <= D)
            solver.Add(z[i] + h[i] <= H)

        for j in range(n):
            solver.Add(solver.Sum([f[i][j] for i in range(n) if i!=j]) * Mh >= z[j])
            
        #funzione obiettivo
        objective = solver.Objective()
        objective.SetCoefficient(D,1)
        objective.SetMinimization()

        #soluzione
        solver.SetTimeLimit(amount_time)
        status = solver.Solve()
        t= solver.wall_time()*0.001
        
        s = BP.Solution(self.bp)
        for i in range(len(self.bp.items)):
            rt = False if r[i].solution_value() == 0 else True
            s.pushItem(self.bp.items[i],0,BP.Point(x[i].solution_value(),y[i].solution_value(),0),rt,False)
        return s,t,n,status
    
    def BuildSolution(self,s):
        print s
    
    def plotSolution(self,s):
        BP_plot.plotSolution(s)
    
    def plot3Dsolution(self,s):
        BP_plot.plot3DSolution(s,0)

    def feasible(self):
        return self.bp.bins[0].height*self.bp.bins[0].width >= sum([self.bp.items[i].width*self.bp.items[i].height for i in range(len(self.bp.items))])

In [6]:
def calculate_result(istance,p_result):
    model = model_bin_packing(istance[0],istance[1])
    s,t,n,status=model.buildModel()
    s.computeObj()
    mod_obj =round(s.obj,2)
    eu_obj =round(euristic(istance[0],istance[1]),2)
    Time =round(t,3)
    n_items =n
    p_result[istance[3]]=[mod_obj,eu_obj,Time,n_items,istance[2],status]
    

In [7]:
ist = []
df = pd.read_excel('istanza.xlsx')
for i in range(len(df)):
    ist.append(istanza(intervallo(df.w_a[i],df.w_b[i]),intervallo(df.d_a[i],df.d_b[i]),i))

In [8]:
models_x_istance = 100
pre = []

for i in ist:
    for j in range(models_x_istance):
        items,bins=tc_sfida(i)
        pre.append([items,bins,i.index,((i.index*100)+j)])

In [9]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
pre =sorted(pre, key=lambda x: len(x[0]))
pre = list(chunks(pre, multiprocessing.cpu_count()))

In [11]:
tm=time.time()
manager = multiprocessing.Manager()
p_result = manager.dict()
for i in pre:
    jobs = []
    for j in i:
        p = multiprocessing.Process(target=calculate_result, args=(j,p_result))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
print time.time()-tm

14408.221051


In [12]:
result=p_result.values()

mod_obj = [n_it[0] for n_it in result]
eu_obj = [n_it[1] for n_it in result]
Time = [n_it[2] for n_it in result]
n_items = [n_it[3] for n_it in result]
istance = [n_it[4] for n_it in result]
optime = [n_it[5] for n_it in result]

In [13]:
tot_df =pd.DataFrame({'n_items': n_items, 'Eu_obj': eu_obj,'Mod_obj': mod_obj, 'Time': Time, 'Istance': istance, 'Optime': optime})

for i in range(len(ist)):
    result_ist =tot_df[(tot_df['Istance'] == i)]
    result_ist =result_ist.drop(['Istance'],axis=1)
    result_ist.to_excel('result'+str(i)+'.xlsx', encoding='utf8')

tot_df.drop(['Istance'],axis=1)
tot_df.to_excel('result_tot.xlsx', encoding='utf8')

In [14]:
tot_df

,Eu_obj,Istance,Mod_obj,Optime,Time,n_items
0,8.18,0,8.18,0,17.048,6
1,6.75,0,6.66,0,77.534,7
2,7.84,0,7.41,1,305.372,9
3,3.16,0,3.16,0,0.029,3
4,2.60,0,2.60,0,0.081,3
5,5.53,0,5.22,0,121.248,7
6,10.17,0,9.50,1,306.108,10
7,5.00,0,4.79,0,3.770,6
8,4.51,0,4.51,0,0.026,3
9,8.14,0,7.00,0,1.464,6


In [15]:
send_mail()